In [18]:
import requests
from dotenv import load_dotenv
import os
from collections import defaultdict
import json

In [19]:
def load_env():
    load_dotenv()
    TICKETMASTER_API_KEY = os.getenv("TICKETMASTER_API_KEY")
    BASE_URL = "https://app.ticketmaster.com/discovery/v2/events.json"
    return TICKETMASTER_API_KEY, BASE_URL

In [20]:
def get_events_helper(city, size=200):
    TICKETMASTER_API_KEY, BASE_URL = load_env()
    params = {
        "apikey": TICKETMASTER_API_KEY,
        "city": city,
        "size": size
    }
    response = requests.get(BASE_URL, params=params)
    response.raise_for_status()
    data = response.json()

    if '_embedded' not in data:
        return []
    events = data.get('_embedded', []).get('events', [])

    event_list = []
    for event in events:
        try:
            name = event.get('name')
            date = event.get('dates', {}).get('start', {}).get('localDate')
            time = event.get('dates', {}).get('start', {}).get('localTime')

            venue = event.get('_embedded', {}).get('venues', [{}])[0]
            venue_name = venue.get('name')
            address = venue.get('address', {}).get('line1')
            city = venue.get('city', {}).get('name')
            state = venue.get('state', {}).get('name')

        # Only add events with all mandatory fields
            if not all([name, date, time, venue_name, address, city, state]):
                continue
            event_details = {
                'name': name,
                'date': date,
                'time': time,
                'venue': {
                    'name': venue_name,
                    'address': address,
                    'city': city,
                    'state': state
                }
            }
            classifications = event.get('classifications')
            if classifications and isinstance(classifications, list) and len(classifications) > 0:
                classification = classifications[0]
                segment = classification.get('segment', {}).get('name', 'N/A')
                genre = classification.get('genre', {}).get('name', 'N/A')
                subgenre = classification.get('subGenre', {}).get('name', 'N/A')
                event_details['classification'] = {
                    'segment': segment,
                    'genre': genre,
                    'subgenre': subgenre
                }
            else:
                continue
            event_list.append(event_details)
        except Exception as e:
            #print(f"Skipped event due to error: {e}")
            continue
    return event_list

In [21]:
def get_events_info_from_city(city):
    events =  get_events_helper(city)
    #print(events)
    merged_events = {}
    for event in events:
        key = (
            event['venue']['name'],
            event['venue']['address'],
            event['venue']['city'],
            event['venue']['state'],
            event['venue'].get('country', 'N/A'),
            event['date'],
            event['time'],
        )
        if key not in merged_events:
            merged_events[key] = {**event, "name": [event["name"]]}
        else:
            merged_events[key]["name"].append(event["name"])
    final_event_list = list(merged_events.values())
    
    segmented_events = defaultdict(lambda: defaultdict(list))
    
    for event in events:
        segment = event.get('classification', {}).get('segment', 'Unknown')
        genre = event.get('classification', {}).get('genre', 'Unknown')
        # Remove classification from event for compactness, if you want:
        compact_event = event.copy()
        if 'classification' in event and 'subgenre' in event['classification']:
            compact_event['subgenre'] = event.get('classification').get('subgenre')
        compact_event.pop('classification', None)
        segmented_events[segment][genre].append(compact_event)
           
    # To convert to normal dicts (optional, for pretty printing or export)
    nested_event_dict = json.loads(json.dumps(segmented_events))
    nested_event_dict.pop('Undefined', {})    
        
    return nested_event_dict


In [22]:
#get_events_info_from_city("saint louis")